In [1]:
import pandas as pd
import numpy as np
from functions_NLM import estimate_nested_logit, simulate_choice
import matplotlib.pyplot as plt

<h2> CNLM with alpha as variable </h2>

In [2]:
# Define log-likelihood function for telephone data
# beta will be beta = ["ASC_1", "ASC_3", "ASC_4", "ASC_5",
#                       "BETA_COST", "lambda_measured", "lambda_flat", "alpha_3_with_measured"]

# before ["ASC_CAR", "ASC_SM", "BETA_TT", "BETA_TC", "lambda_CAR_TRAIN", "lambda_SM_TRAIN", "base_alpha_TRAIN_WITH_CAR"]

# DEFINE MODEL STRUCTURE AND LIKELIHOOD FUNCTION
def log_likelihood_telephone_CNLM(beta, data):
    alpha_3_with_measured = np.exp(beta[7]) / (1 + np.exp(beta[7]))
    alpha_3_with_flat = 1 - alpha_3_with_measured 
    # alpha_3_with_measured should be between 0 and 1
    if alpha_3_with_measured < 0: 
        alpha_3_with_measured = 0

    if alpha_3_with_measured > 1: 
        alpha_3_with_measured = 1

            
    # Define utility functions
    data['U_1'] = beta[0] + beta[4] * data['logcost1']
    data['U_2'] = beta[4] * data['logcost2']
    data['U_3'] = beta[1] + beta[4] * data['logcost3']
    data['U_4'] = beta[2] + beta[4] * data['logcost4']
    data['U_5'] = beta[3] + beta[4] * data['logcost5']

    # combined utility terms inside nests
    data['log_U_measured_3'] = np.log((alpha_3_with_measured * data['avail3'] * np.exp(data['U_3'])) ** (1 / beta[5]) + \
                             data['avail1'] * np.exp(data['U_1']) ** (1 / beta[5]) + \
                             data['avail2'] * np.exp(data['U_2']) ** (1 / beta[5]))
    data['log_U_flat_3'] = np.log((alpha_3_with_flat * data['avail3'] * np.exp(data['U_3'])) ** (1 / beta[6]) + \
                            data['avail4'] * np.exp(data['U_4']) ** (1 / beta[6]) + \
                            data['avail5'] * np.exp(data['U_5']) ** (1 / beta[6]))

    # Nest probabilities
    data['log_P_nest_measured_3'] = data['log_U_measured_3'] * beta[5] - \
                                np.log(np.exp(data['log_U_measured_3']) ** beta[5] + \
                                       np.exp(data['log_U_flat_3']) ** beta[6])
    data['log_P_nest_flat_3'] = np.log(1 - np.exp(data['log_P_nest_measured_3']))

    # Within nest probabilities
    data['log_P_1_in_measured_3'] = np.log(data['avail1']) + data['U_1'] / beta[5] - data['log_U_measured_3']
    data['log_P_3_in_measured_3'] = (np.log(data['avail3']) + np.log(alpha_3_with_measured) + \
                                     data['U_3']) / beta[5] - data['log_U_measured_3']
    data['log_P_2_in_measured_3'] = np.log(1 - np.exp(data['log_P_1_in_measured_3']) - \
                                           np.exp(data['log_P_3_in_measured_3']))
    data['log_P_4_in_flat_3'] = np.where(data['avail4'] == 0, -np.inf,
                                     np.log(data['avail4']) + data['U_4'] / beta[6] - data['log_U_flat_3'])
    data['log_P_3_in_flat_3'] = (np.log(data['avail3']) + np.log(alpha_3_with_flat) + \
                                 data['U_3']) / beta[6] - data['log_U_flat_3']
    data['log_P_5_in_flat_3'] = np.where(data['avail5'] == 0, -np.inf, 
                                         np.log(1 - np.exp(data['log_P_4_in_flat_3']) - \
                                                np.exp(data['log_P_3_in_flat_3'])))

    # Full probabilities
    data['P_1'] = np.exp(data['log_P_nest_measured_3'] + data['log_P_1_in_measured_3'])
    data['P_2'] = np.exp(data['log_P_nest_measured_3'] + data['log_P_2_in_measured_3'])
    data['P_3'] = np.exp(data['log_P_nest_measured_3'] + data['log_P_3_in_measured_3']) + \
                np.exp(data['log_P_nest_flat_3'] + data['log_P_3_in_flat_3'])
    data['P_4'] = np.exp(data['log_P_nest_flat_3'] + data['log_P_4_in_flat_3'])
    data['P_5'] = np.exp(data['log_P_nest_flat_3'] + data['log_P_5_in_flat_3'])

    # Calculate probability for chosen alternative for each row
    data['P'] = (data['choice'] == 1) * data['P_1'] + \
                (data['choice'] == 2) * data['P_2'] + \
                (data['choice'] == 3) * data['P_3'] + \
                (data['choice'] == 4) * data['P_4'] + \
                (data['choice'] == 5) * data['P_5']

    # Calculate log-likelihood 
    LL = data['P'].apply(np.log).sum()

    return -LL  # We minimize negative log-likelihood

In [3]:
# Load data
data = pd.read_csv('../data/telephone.dat', sep='\t')

data['logcost1'] = np.log(data['cost1'])
data['logcost2'] = np.log(data['cost2'])
data['logcost3'] = np.log(data['cost3'])
data['logcost4'] = np.log(data['cost4'])
data['logcost5'] = np.log(data['cost5'])

# Define model parameters
beta = np.array([0, 0, 0, 0, 0, 1, 1, 0])
# lambda_n = 1 / mu_n is a measure of the degree of independence in unobserved utility among
# the alternatives in nest n.
# It should be between 0 and 1 with lambda_n = 1 indicating full independence.
beta_names = ["ASC_1", "ASC_3", "ASC_4", "ASC_5", "BETA_COST", "lambda_measured", "lambda_flat", "alpha_3_with_measured"]


In [4]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import t
import scipy.sparse as sp

In [5]:
# We have to change the function estimate_nested_logit because we have to add bounds for alpha_3_with_measured
def estimate_nested_logit_CNLM(data, beta_initial, beta_names, log_likelihood_function):
    """
    Estimate parameters for a nested logit model using maximum likelihood estimation.

    Args:
    - data (DataFrame): Input dataset containing variables needed for the model.
    - beta_initial (array-like): Initial guess for model parameters.
    - beta_names (list): Names of model parameters.
    - log_likelihood_function (function): Function that calculates the log-likelihood of the model. 

    Returns:
    - result (OptimizeResult): Result object from scipy.optimize.minimize containing optimization results.
    - se (array-like): Robust asymptotic standard errors of parameter estimates.
    - t_stat (array-like): t-statistics of parameter estimates.
    - p_value (array-like): p-values of parameter estimates.
    """

    # Run the model
    result = minimize(log_likelihood_function, x0 = beta_initial, args = data, method='L-BFGS-B',
    bounds = ((None, None), (None, None), (None, None), (None, None), (None, None), (None, None),
               (None, None), (0, 1)))

    # Calculate Hessian matrix
    hessian_inv = result.hess_inv.todense()

    # Calculate robust asymptotic standard errors
    se = np.sqrt(np.diag(hessian_inv))

    # Calculate t-statistics
    t_stat = result.x / se

    # Calculate p-values
    p_value = (1 - t.cdf(np.abs(t_stat), len(data) - len(beta_initial))) * 2

    # Calculate AIC
    log_likelihood_value = -result.fun
    k = len(beta_initial)
    n = len(data)
    aic = 2 * k - 2 * log_likelihood_value

    # Calculate BIC
    bic = np.log(n) * k - 2 * log_likelihood_value

    # Create DataFrame to store results
    results_df = pd.DataFrame({
        "Parameter": beta_names,
        "Estimate": result.x,
        "Robust Asymptotic SE": se,
        "t-statistic": t_stat,
        "p-value": p_value
    })

    print("Optimization Results:")
    print(results_df)
    print("AIC:", aic)
    print("BIC:", bic)

    return result, se, t_stat, p_value, aic, bic


In [6]:
# Estimate parameters
result, se, t_stat, p_value, aic, bic  = estimate_nested_logit_CNLM(data, beta,
                                                                     beta_names, log_likelihood_telephone_CNLM)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Optimization Results:
               Parameter  Estimate  Robust Asymptotic SE  t-statistic  \
0                  ASC_1 -1.426263              0.501990    -2.841216   
1                  ASC_3  2.003235              0.506590     3.954352   
2                  ASC_4  2.147718              3.059635     0.701952   
3                  ASC_5  3.740691              1.470948     2.543048   
4              BETA_COST -3.198239              0.876837    -3.647473   
5        lambda_measured  2.172737              0.729996     2.976370   
6            lambda_flat  1.894093              2.576903     0.735027   
7  alpha_3_with_measured  1.000000              1.000000     1.000000   

    p-value  
0  0.004710  
1  0.000090  
2  0.483092  
3  0.011342  
4  0.000298  
5  0.003083  
6  0.462728  
7  0.317878  
AIC: 962.6011872954471
BIC: 995.1855435682503


---
---
---
---
---

<h2> CNLM with fixed alpha </h2>

In [7]:
# Define log-likelihood function for telephone data
# beta will be beta = ["ASC_1", "ASC_3", "ASC_4", "ASC_5",
#                       "BETA_COST", "lambda_measured", "lambda_flat"]


# DEFINE LIKELIHOOD FUNCTION
def log_likelihood_telephone_CNLM2(beta, data):
    alpha_3_with_measured = 0.5
    alpha_3_with_flat = 1 - alpha_3_with_measured

    # Define utility functions
    data['U_1'] = beta[0] + beta[4] * data['logcost1']
    data['U_2'] = beta[4] * data['logcost2']
    data['U_3'] = beta[1] + beta[4] * data['logcost3']
    data['U_4'] = beta[2] + beta[4] * data['logcost4']
    data['U_5'] = beta[3] + beta[4] * data['logcost5']

    # combined utility terms inside nests
    data['log_U_measured_3'] = np.log((alpha_3_with_measured * data['avail3'] * np.exp(data['U_3'])) ** (1 / beta[5]) + \
                             data['avail1'] * np.exp(data['U_1']) ** (1 / beta[5]) + \
                             data['avail2'] * np.exp(data['U_2']) ** (1 / beta[5]))
    data['log_U_flat_3'] = np.log((alpha_3_with_flat * data['avail3'] * np.exp(data['U_3'])) ** (1 / beta[6]) + \
                            data['avail4'] * np.exp(data['U_4']) ** (1 / beta[6]) + \
                            data['avail5'] * np.exp(data['U_5']) ** (1 / beta[6]))

    # Nest probabilities
    data['log_P_nest_measured_3'] = data['log_U_measured_3'] * beta[5] - \
                                np.log(np.exp(data['log_U_measured_3']) ** beta[5] + \
                                       np.exp(data['log_U_flat_3']) ** beta[6])
    data['log_P_nest_flat_3'] = np.log(1 - np.exp(data['log_P_nest_measured_3']))

    # Within nest probabilities
    data['log_P_1_in_measured_3'] = np.log(data['avail1']) + data['U_1'] / beta[5] - data['log_U_measured_3']
    data['log_P_3_in_measured_3'] = (np.log(data['avail3']) + np.log(alpha_3_with_measured) + \
                                     data['U_3']) / beta[5] - data['log_U_measured_3']
    data['log_P_2_in_measured_3'] = np.log(1 - np.exp(data['log_P_1_in_measured_3']) - \
                                           np.exp(data['log_P_3_in_measured_3']))
    data['log_P_4_in_flat_3'] = np.where(data['avail4'] == 0, -np.inf,
                                     np.log(data['avail4']) + data['U_4'] / beta[6] - data['log_U_flat_3'])
    data['log_P_3_in_flat_3'] = (np.log(data['avail3']) + np.log(alpha_3_with_flat) + \
                                 data['U_3']) / beta[6] - data['log_U_flat_3']
    data['log_P_5_in_flat_3'] = np.where(data['avail5'] == 0, -np.inf, 
                                         np.log(1 - np.exp(data['log_P_4_in_flat_3']) - \
                                                np.exp(data['log_P_3_in_flat_3'])))

    # Full probabilities
    data['P_1'] = np.exp(data['log_P_nest_measured_3'] + data['log_P_1_in_measured_3'])
    data['P_2'] = np.exp(data['log_P_nest_measured_3'] + data['log_P_2_in_measured_3'])
    data['P_3'] = np.exp(data['log_P_nest_measured_3'] + data['log_P_3_in_measured_3']) + \
                np.exp(data['log_P_nest_flat_3'] + data['log_P_3_in_flat_3'])
    data['P_4'] = np.exp(data['log_P_nest_flat_3'] + data['log_P_4_in_flat_3'])
    data['P_5'] = np.exp(data['log_P_nest_flat_3'] + data['log_P_5_in_flat_3'])

    # Calculate probability for chosen alternative for each row
    data['P'] = (data['choice'] == 1) * data['P_1'] + \
                (data['choice'] == 2) * data['P_2'] + \
                (data['choice'] == 3) * data['P_3'] + \
                (data['choice'] == 4) * data['P_4'] + \
                (data['choice'] == 5) * data['P_5']

    # Calculate log-likelihood 
    LL = data['P'].apply(np.log).sum()

    return -LL  # We minimize negative log-likelihood

In [8]:
# Load data
data = pd.read_csv('./data/telephone.dat', sep='\t')

data['logcost1'] = np.log(data['cost1'])
data['logcost2'] = np.log(data['cost2'])
data['logcost3'] = np.log(data['cost3'])
data['logcost4'] = np.log(data['cost4'])
data['logcost5'] = np.log(data['cost5'])

# Define model parameters
beta = np.array([0, 0, 0, 0, 0, 1, 1])
# lambda_n = 1 / mu_n is a measure of the degree of independence in unobserved utility among
# the alternatives in nest n.
# It should be between 0 and 1 with lambda_n = 1 indicating full independence.
beta_names = ["ASC_1", "ASC_3", "ASC_4", "ASC_5", "BETA_COST", "lambda_measured", "lambda_flat"]


FileNotFoundError: [Errno 2] No such file or directory: './data/telephone.dat'

In [ ]:
# Estimate parameters
result, se, t_stat, p_value, aic, bic  = estimate_nested_logit(data, beta, beta_names, log_likelihood_telephone_CNLM2)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encount

Optimization Results:
         Parameter  Estimate  Robust Asymptotic SE  t-statistic       p-value
0            ASC_1 -1.635754              0.496645    -3.293610  1.071442e-03
1            ASC_3  2.199670              0.556739     3.950990  9.104589e-05
2            ASC_4  2.426402              1.316582     1.842956  6.602838e-02
3            ASC_5  4.084728              1.053189     3.878438  1.217014e-04
4        BETA_COST -3.354164              0.582177    -5.761416  1.598521e-08
5  lambda_measured  2.541826              0.624433     4.070612  5.587709e-05
6      lambda_flat  1.957885              0.714936     2.738544  6.429389e-03
AIC: 961.2124055418653
BIC: 989.7237172805682


---
---
---
---
---

<h2> Applying DIB algorithm </h2>

In [ ]:
data_logcost = data[['logcost1', 'logcost2', 'logcost3', 'logcost4', 'logcost5']]

# Function to compare rows with a reference row
def count_same_rows(df):
    row_counts = {}

    for index, row in df.iterrows():
        # Convert the row to a tuple to make it hashable
        row_tuple = tuple(row)
        
        # Count the occurrences of the row in the dataframe
        if row_tuple in row_counts:
            row_counts[row_tuple] += 1
        else:
            row_counts[row_tuple] = 1
            
    return row_counts

# Count occurrences of each row
row_counts = count_same_rows(data_logcost)

# Add a new column with probabilities
total_rows = len(data_logcost)
data_logcost['probability'] = data_logcost.apply(lambda row: row_counts[tuple(row)] / total_rows, axis=1)
data_logcost.head()


/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_1919/1840364846.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_logcost['probability'] = data_logcost.apply(lambda row: row_counts[tuple(row)] / total_rows, axis=1)


,logcost1,logcost2,logcost3,logcost4,logcost5,probability
0,1.761300,1.754404,2.545531,13.815511,3.147595,0.002304
1,1.258461,1.754404,2.507972,13.815511,3.147595,0.002304
2,1.627278,1.754404,2.439735,13.815511,3.342155,0.002304
3,1.558145,1.754404,2.347558,13.815511,3.342155,0.002304
4,2.145931,1.953028,2.662355,13.815511,3.342155,0.002304


In [ ]:
p_x = data_logcost['probability'].values
p_y_given_x = data[['P_1', 'P_2', 'P_3', 'P_4', 'P_5']].values
p_xy = p_x[:, np.newaxis] * p_y_given_x

# Normalize p_xy 
p_xy /= p_xy.sum()

# Define epsilon value
epsilon = 1e-20

# Add epsilon to elements equal to 0
p_xy[p_xy == 0] += epsilon

<h2> Modified version of the DIB algorithm where the maximum number of clusters is the number of alternatives, not the number of individuals </h2>

In [ ]:
from functions_geom_DIB import geom_DIB_on_alternatives

In [ ]:
q_t_given_x, q_t, q_y_given_t = geom_DIB_on_alternatives(p_xy, beta=5000, max_iter=5000, threshold=1e-10)

Iteration: 1 out of 5000
Iteration: 2 out of 5000
Iteration: 3 out of 5000
Iteration: 4 out of 5000
Iteration: 5 out of 5000
Iteration: 6 out of 5000
Iteration: 7 out of 5000
Iteration: 8 out of 5000
Iteration: 9 out of 5000
Iteration: 10 out of 5000
Iteration: 11 out of 5000


In [ ]:
# Calculate the number of clusters
column_sum = np.sum(q_t_given_x, axis=0)
num_clusters = np.count_nonzero(column_sum)
print("Number of clusters:", num_clusters)

Number of clusters: 5


In [ ]:
# Create new column choice_nest which is 1 if choice= 1 or 2, and 2 otherwise
data['choice_nest'] = np.where(data['choice'].isin([1, 2]), 1, 2)
data['cluster'] = np.argmax(q_t_given_x, axis=1)
data['cluster'].value_counts()

cluster
0    132
3    114
1     99
2     54
4     35
Name: count, dtype: int64

In [ ]:
data['choice_nest'].value_counts()

choice_nest
2    238
1    196
Name: count, dtype: int64

In [ ]:
# number of each alternative 1, 2, 3, 4, 5 in each cluster 
cluster_counts = data.groupby(['cluster', 'choice']).size().unstack(fill_value=0)
cluster_counts

choice,1,2,3,4,5
cluster,,,,,
0,21,58,53,0,0
1,11,34,40,0,14
2,3,10,29,3,9
3,37,20,52,0,5
4,1,1,4,0,29


In [ ]:
data['max_proba'] = data[['P_1', 'P_2', 'P_3', 'P_4', 'P_5']].idxmax(axis=1).str[-1].astype(int)
cluster_counts2 = data.groupby(['cluster', 'max_proba']).size().unstack(fill_value=0)
cluster_counts2

max_proba,1,2,3,4,5
cluster,,,,,
0,0,0,132,0,0
1,0,56,43,0,0
2,0,0,43,2,9
3,21,35,58,0,0
4,0,0,0,0,35


In [ ]:
data['simulated_choice'] = data.apply(simulate_choice, axis=1)
cluster_counts3 = data.groupby(['cluster', 'simulated_choice']).size().unstack(fill_value=0)
cluster_counts3

simulated_choice,1,2,3,4,5
cluster,,,,,
0,19,41,72,0,0
1,13,39,38,0,9
2,4,11,24,5,10
3,27,41,42,0,4
4,1,1,9,1,23


In [ ]:
# betas = np.linspace(0, 50, 51)
# # Initialize an empty list to store the number of clusters
# num_clusters_list = []

# # Iterate over each beta value
# for beta in betas:
#     # Run iterative_algorithm to obtain q_t_given_x
#     q_t_given_x, _, _ = geom_DIB_on_alternatives(p_xy, max_iter=5000, beta=beta, threshold=1e-4)
    
#     # Calculate the number of clusters
#     column_sum = np.sum(q_t_given_x, axis=0)
#     num_clusters = np.count_nonzero(column_sum)
    
#     # Append the number of clusters to the list
#     num_clusters_list.append(num_clusters)

# # Plot the number of clusters against beta values
# plt.plot(betas, num_clusters_list)
# plt.xlabel('Beta')
# plt.ylabel('Number of Clusters')
# plt.title('Number of Clusters vs. Beta')
# plt.grid(True)
# plt.show()